# CONTENT BASED MOVIE RECOMMENDATION SYSTEM

Importing necessary libraries

In [ ]:
import numpy as np
import pandas as pd
import ast

Loading movies, credits dataset

In [ ]:
movies=pd.read_csv('tmdb_5000_movies.csv',encoding='utf-8')
credits=pd.read_csv('tmdb_5000_credits.csv',encoding='utf-8')

ParserError: ignored

In [ ]:
movies.head()

In [ ]:
credits.head()

Merging datasets based on movie titles

In [ ]:
movies=movies.merge(credits,on='title')

In [ ]:
movies.shape

In [ ]:
movies.head(1)

In [ ]:
movies.info()

Selecting the necessary columns

In [ ]:
movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]

Preprocessing

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

In [ ]:
def convert(obj):
  Listt=[]
  for i in ast.literal_eval(obj): #converting string to list in order to iterate through the list
    Listt.append(i['name'])
  return Listt


In [ ]:
movies['genres']=movies['genres'].apply(convert)
movies['genres']

In [ ]:
movies['keywords']=movies['keywords'].apply(convert)
movies['keywords']

In [ ]:
movies.head()

In [ ]:
def cast3(obj):
  Listt=[]
  counter=0
  for i in ast.literal_eval(obj): #converting string to list in order to iterate through the list
    if counter!=3:
      Listt.append(i['name'])
      counter+=1
    else:
      break
  return Listt


In [ ]:
movies['cast']=movies['cast'].apply(cast3) #get top 3 actors of the film

In [ ]:
movies.head()

In [ ]:
#extracting directer name
def get_director(obj):
  Listt=[]
 
  for i in ast.literal_eval(obj): #converting string to list in order to iterate through the list
    if i['job']=='Director':
      Listt.append(i['name'])
      break
  return Listt

In [ ]:
movies['crew']=movies['crew'].apply(get_director)

In [ ]:
movies.head()

In [ ]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [ ]:
movies.head()

In [ ]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies.head()

In [ ]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']


In [ ]:
movies.head()

In [ ]:
new_movies=movies[['movie_id','title','tags']]


In [ ]:
new_movies['tags']=new_movies['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_movies['tags'][0]

In [ ]:
new_movies['tags']=new_movies['tags'].apply(lambda x:x.lower()) #converting all the text to lowercase

In [ ]:
new_movies.head()

In [ ]:
pip install nltk

In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
def stemming(text): #applying stemming to get the root words in the most frequently used words
  y=[]
  for i in text.split():
    y.append(ps.stem(i))

  return " ".join(y)

In [ ]:
new_movies['tags']=new_movies['tags'].apply(stemming)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')
vectors=cv.fit_transform(new_movies['tags']).toarray()

In [ ]:
vectors

In [ ]:
cv.get_feature_names() #most frequentlu occuring words

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity=cosine_similarity(vectors)

In [ ]:
similarity[0]

In [ ]:
def recommend(movie):
  movie_index=new_movies[new_movies['title']==movie].index[0]
  distances=similarity[movie_index]
  movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]

  for i in movies_list:
    print(new_movies.iloc[i[0]].title)
    

In [ ]:
recommend('Avatar')

# New Section

In [ ]:
import pickle

In [ ]:
pickle.dump(new_movies,open('movies.pkl','wb'))

In [ ]:
#pickle.dump(new_movies,open('movies_dict.pkl','wb'))

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))